<a href="https://colab.research.google.com/github/austinjhunt/assign03-50-hunt/blob/main/churn_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>